<a href="https://colab.research.google.com/github/Francimaria/Sistemas_de_Multiplos_Classificadores/blob/main/exercicio1_sgh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install  deslib

     |████████████████████████████████| 153kB 9.0MB/s 


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import Perceptron
from sklearn.ensemble import BaseEnsemble
from sklearn.linear_model import SGDClassifier
from scipy import stats

from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from deslib.static import Oracle

### **Questão 2**

Use as mesma bases de dados e os mesmos folds da questão anterior e, para cada base: - use o SGH para gerar o pool de classificadores no conjunto de treinamento; - calcule o Oracle do pool no conjunto de teste; - verifique quantas instâncias por classe foram incorretamente classificadas; - verifique quantos hiperplanos por classe foram gerados. Analise os resultados coletados.

In [4]:
file_1_name = "https://raw.githubusercontent.com/Francimaria/Sistemas_de_Multiplos_Classificadores/main/dataset/messidor_features.csv"
file_2_name = "https://raw.githubusercontent.com/Francimaria/Sistemas_de_Multiplos_Classificadores/main/dataset/breast-cancer-wisconsin.csv"
file_3_name = "https://raw.githubusercontent.com/Francimaria/Sistemas_de_Multiplos_Classificadores/main/dataset/data_banknote_authentication.csv"
file_4_name = "https://raw.githubusercontent.com/Francimaria/Sistemas_de_Multiplos_Classificadores/main/dataset/winequality-white.csv"
file_5_name = "https://raw.githubusercontent.com/Francimaria/Sistemas_de_Multiplos_Classificadores/main/dataset/Frogs_MFCCs.csv"

df1 = pd.read_csv(file_1_name, header=None,skiprows=1)
df2 = pd.read_csv(file_2_name, header=None)
df3 = pd.read_csv(file_3_name, header=None)
df4 = pd.read_csv(file_4_name, header=None,skiprows=1, sep=";")
df5 = pd.read_csv(file_5_name, header=None,skiprows=1)

**Pré-processamento**

In [5]:
print("Shape: ", df1.shape)
print(df1[df1.columns[-1]].value_counts())

print("Shape: ", df2.shape)
print("0 for benign, 1 for malignant")
df2[df2.columns[-1]].replace([2, 4], [0, 1], inplace=True)
print(df2[df2.columns[-1]].value_counts())

# Remove ID
df2 = df2.drop([df2.columns[0]], axis=1)

print("Shape: ", df3.shape)
print(df3[df3.columns[-1]].value_counts())

print("Shape: ", df4.shape)
print("Replace classes [3,4,..,9] to [0,1,...6]")
df4[df4.columns[-1]].replace([3,4,5,6,7,8,9], [0,1,2,3,4,5,6], inplace=True)
print(df4[df4.columns[-1]].value_counts())

# Remover ID, gênero, e espécie (apenas families é usado como classe)
df5 = df5.drop([23,24,25], axis=1)

print("Replace classes [Leptodactylidae, Hylidae, Dendrobatidae, Bufonidae] to [0,1,2,3]")
print("Shape: ", df5.shape)
df5[df5.columns[-1]].replace(['Leptodactylidae', 'Hylidae', 'Dendrobatidae', 'Bufonidae'], [0,1,2,3], inplace=True)
print(df5[df5.columns[-1]].value_counts())

Shape:  (1151, 20)
1    611
0    540
Name: 19, dtype: int64
Shape:  (683, 11)
0 for benign, 1 for malignant
0    444
1    239
Name: 10, dtype: int64
Shape:  (1372, 5)
0    762
1    610
Name: 4, dtype: int64
Shape:  (4898, 12)
Replace classes [3,4,..,9] to [0,1,...6]
3    2198
2    1457
4     880
5     175
1     163
0      20
6       5
Name: 11, dtype: int64
Replace classes [Leptodactylidae, Hylidae, Dendrobatidae, Bufonidae] to [0,1,2,3]
Shape:  (7195, 23)
0    4420
1    2165
2     542
3      68
Name: 22, dtype: int64


In [6]:
data = []
classe = []
df = [df1,df2,df3,df4,df5]

# Padronização dos dados na mesma escala
for d in df:
  x = d.drop([d.columns[-1]], axis=1)
  x = np.array(preprocessing.scale(x))  
  y = np.array(d[d.columns[-1]])
  
  data.append(x)
  classe.append(y)

K-fold inicialização

In [7]:
#random_state: Using an int will produce the same results across different calls. 
kf = StratifiedKFold(n_splits=5, random_state=5)

n_estimators = np.arange(10, 110, 10)
print(n_estimators)

base_estimator=Perceptron()
print(base_estimator)


[ 10  20  30  40  50  60  70  80  90 100]
Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)


In [8]:
from sgh import SGH

In [17]:
def run_sgh(pool_classifiers, X, y, ds_name): 
  score = []
  oracle_score = []
  estimators = []
  qtds = []
  misc = []
  misc_std = []
  classes = np.unique(y)
  
  for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    pool_classifiers.fit(X_train, y_train)

    est = pool_classifiers.n_estimators
    estimators.append(est)

    y_pred = pool_classifiers.predict(X_test)

    score.append(accuracy_score(y_test,y_pred))
   
    qtds.append(count_erro(y_test, y_pred, classes))
    
    oracle = Oracle(pool_classifiers).fit(X_train, y_train)

    oracle_score.append(oracle.score(X_test, y_test))  

  misc.append(np.mean(qtds, axis=0))
  misc_std.append(np.std(qtds, axis=0))

  return score, oracle_score, estimators, misc, misc_std

In [18]:
def count_erro(y_test, y_pred, classes):
  y_test = pd.Series(y_test)
  y_pred = pd.Series(y_pred)
  
  classes = sorted(classes)

  misclassified = np.zeros(((len(classes)),), dtype=int)

  for i in classes:
    qtd = 0

    for j in range(len(y_test)):
      if (y_test[j] == i and y_test[j] != y_pred[j]):
        qtd +=1
    misclassified[i] = qtd
  return misclassified

In [19]:
# Datafile para salvar os resultados
df_result = pd.DataFrame(columns = ['Modelo', 'Dataset','estimators','std', 'Score_mean','Score_std', 'Oracle_mean', 'Oracle_std'])
#Executar para todos os conjuntos de dados
for i in range(len(data)):
  clf = SGH(base_estimator=base_estimator)   
  score, oracle, estimators, mean, std = run_sgh(clf, data[i], classe[i], i)
  print("média")
  print(mean)
  print("std")
  print(std)

  new_row = {'Modelo':"SGH",
            'Dataset': (i+1), 
            'estimators': np.mean(estimators),
            'std': np.std(estimators),
            'Score_mean': "%0.5f" % (np.mean(score)),
            'Score_std': "%0.5f" % (np.std(score)),
            'Oracle_mean': "%0.5f" % (np.mean(oracle)),
            'Oracle_std': "%0.5f" % (np.std(oracle))}
  df_result = df_result.append(new_row, ignore_index=True)
  
df_result.to_csv("/content/sgh_result.csv", index=False)
df_result

média
[array([85.6, 30.2])]
std
[array([ 7.73563184, 12.60793401])]
média
[array([85.6, 44.6])]
std
[array([1.356466  , 1.95959179])]
média
[array([72.8,  5.2])]
std
[array([5.23067873, 1.32664992])]
média
[array([3.600e+00, 3.240e+01, 2.496e+02, 4.252e+02, 1.746e+02, 2.540e+01,
       2.000e-01])]
std
[array([ 0.8       ,  0.48989795, 83.80119331, 28.60349629,  2.8       ,
        3.66606056,  0.4       ])]
média
[array([727.6, 192.8,  64. ,  13.4])]
std
[array([303.80625405, 146.27426294,  51.93072308,   0.48989795])]


,Modelo,Dataset,estimators,std,Score_mean,Score_std,Oracle_mean,Oracle_std
0,SGH,1,6.2,1.6,0.49692,0.02468,0.99913,0.00173
1,SGH,2,3.8,0.4,0.04680,0.02139,0.99854,0.00292
2,SGH,3,3.0,0.0,0.71572,0.02360,1.00000,0.00000
3,SGH,4,21.6,4.8,0.07000,0.11657,0.99878,0.00245
4,SGH,5,10.6,2.8,0.30660,0.15001,0.97832,0.04336
